## Intro

This is inspired by 
Article (likas2001probability) Likas, A. Probability density estimation using artificial neural networks Computer physics communications, Elsevier, 2001, 135, 167-175

But rather than estimating the working with a network, we will instead work with its derivitive.
This will let us replace their integration with a derivative.

Note that this method only works for compact supports



They use the PDF is given by $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p) dz}$$
and in their case $h=N(x,p)$  a neural network with weight and bias parameters $p$.
Where $S$ is a compact support. (That means bounded)


But if instead we say $h=\frac{\partial N(x,p)}{\partial x}$,

then $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p)}=\dfrac{\frac{\partial N(x,p)}{\partial x}}{N(max(S),p) - N(min(S), p)}$$

The denominator is ofcourse more complex for non-1D values of S.


The loss function given is the negative log-likelihood of the set of training samples $X$
$$L(p) = -\sum_{\forall x \in X} ln(h(x,p))  + |X| ln(\int_S h(z,p) dx)$$

Which befomes:

$$L(p) = -\sum_{\forall x \in X} log(\frac{\partial N(x,p)}{\partial x})  + |X|(ln(N(max(S),p)-N(min(S),p)) dx$$

In [1]:
using StatsBase
using Distributions

In [2]:
using TensorFlow
using MLDataUtils

In [3]:
using DensityEstimationML



In [4]:
immutable NeuralDensityEstimator
    sess::Session
    
    #Network nodes
    optimizer::Tensor
    t::Tensor
    pdf::Tensor
end

In [5]:
function Distributions.pdf(est::NeuralDensityEstimator, t::Real)
    gr = est.sess.graph
    ts = reshape([t], (1,1))
    run(est.sess, est.pdf, Dict(est.t=>ts))[1]
end

In [25]:
function NeuralDensityEstimator(prob_layer_sizes, support_min, support_max)
    sess = Session(Graph())
    @tf begin
        t = placeholder(Float32, shape=[1, -1])
        smin = constant(reshape([support_min],(1,1)))
        smax = constant(reshape([support_min],(1,1)))
        
        layer_sizes= [1; prob_layer_sizes; 1]
        
        network_fun_stack = Function[Base.identity]       
       
        for ii in 2:length(layer_sizes)
            below_size = layer_sizes[ii-1]
            above_size = layer_sizes[ii]
                       
            Wii = get_variable("W_$ii", [above_size, below_size], Float32)
            
            act_fun = if ii!=length(layer_sizes)
                bii = get_variable("b_$ii", [above_size, 1], Float32)
                z -> nn.sigmoid(Wii*z .+ bii)
            else
                z-> exp(Wii*z)
            end
            push!(network_fun_stack, z->act_fun(network_fun_stack[ii-1](z)))
        end
        
        network = network_fun_stack[end]
        

        
        zsmin = network(smin)
        zsmax = network(smax)
        
        ysmin = min(zsmin,zsmax)
        ysmax = max(zsmin,zsmax)
        
        yt = select(zsmin[end]<zsmax[end], network(t), network(-1.*t))
        
        
        denominator = (ysmax-ysmin) #area
        numerator = gradients(yt,t)
        pdf =numerator/denominator
        
        
        n_points = TensorFlow.shape(t)[2]
        true_loss= -reduce_sum(log(numerator))+ n_points.*log(denominator)
        
        area_loss = (1f0.-denominator)^2
        working_loss = true_loss + 0.1*area_loss
        
        optimizer = train.minimize(train.AdamOptimizer(), working_loss)
        
    end
    
    run(sess, global_variables_initializer())
    
    NeuralDensityEstimator(sess, optimizer, t, pdf)
end

NeuralDensityEstimator

In [26]:
function StatsBase.fit!(estimator::NeuralDensityEstimator, observations;
    epochs = 20)
    
    gr = estimator.sess.graph
    for ii in 1:epochs
        loss_o = run(estimator.sess, 
            [gr["true_loss"], gr["working_loss"],gr["denominator"],
                estimator.optimizer],
            Dict(estimator.t=>observations'))
        println("Epoch $ii: loss: $(loss_o)")
    end
    estimator
end

In [27]:
dataset = GenerateDatasets.likas_1
data = dataset()
est = NeuralDensityEstimator([1024], support(dataset)...)


2017-09-07 15:50:16.676910: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:01:00.0)


LoadError: [91mUndefRefError: access to undefined reference[39m

In [ ]:
fit!(est, data; epochs=1000)

In [22]:
using Plots

using MacroTools
macro plot(ex)
    @capture(ex, (x_, [ys__], tail__ )) 
    labels = repr.(ys)
    ys_expr = Expr(:hvect, ys...)
    labels_expr = Expr(:vect, repr.(ys)...)
    Expr(:call, :plot, x, ys_expr, tail..., Expr(:kw, :labels, labels_expr))
end


plot(x->pdf(est,x), xlims= support(dataset))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 0 
 
 
 10 
 
 
 0.047 
 
 
 0.048 
 
 
 0.049 
 
 
 0.050 
 
 
 
 
 
 
 y1

In [ ]:
dataset = GenerateDatasets.magdon_ismail_and_atiya
data = dataset()
est = NeuralDensityEstimator([1024], -16,16)
fit!(est, data; epochs=10_000)

In [ ]:
?TensorFlow.Ops.select

In [53]:
X = -3:0.01:3
f(x) = x>0?log(x):0.0
plot(X, f.(X), xlims=(-3,3), ylims=(-5,5))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.1436,193.812 40.0455,193.812 40.9475,193.812 41.8494,193.812 42.7514,193.812 43.6533,193.812 44.5553,193.812 45.4573,193.812 46.3592,193.812 47.2612,193.812 
 48.1631,193.812 49.0651,193.812 49.967,193.812 50.869,193.812 51.7709,193.812 52.6729,193.812 53.5748,193.812 54.4768,193.812 55.3787,193.812 56.2807,193.812 
 57.1826,193.812 58.0846,193.812 58.9865,193.812 59.8885,193.812 60.7904,193.812 61.6924,193.812 62.5943,193.812 63.4963,193.812 64.3983,193.812 65.3002,193.812 
 66.2022,193.812 67.1041,193.812 68.0061,193.812 68.908,193.812 69.81,193.812 70.7119,193.812 71.6139,193.812 72.5158,193.812 73.4178,193.812 74.3197,193.812 
 75.2217,193.812 76.1236,193.812 77.0256,193.812 77.9275,193.812 78.8295,193.812 79.7314,193.812 80.6334,193.812 81.5353,193.812 82.4373,193.812 83.3392,193.812 
 84.2412,193.812 85.1432,193.812 86.0451,193.812 86.9471,193.812 87.849,193.812 88.751,193.812 89.6529,193.812 90.5549,193.812 91.4568,193.812 92.3588,193.812 
 93.2607,193.812 94.1627,193.812 95.0646,193.812 95.9666,193.812 96.8685,193.812 97.7705,193.812 98.6724,193.812 99.5744,193.812 100.476,193.812 101.378,193.812 
 102.28,193.812 103.182,193.812 104.084,193.812 104.986,193.812 105.888,193.812 106.79,193.812 107.692,193.812 108.594,193.812 109.496,193.812 110.398,193.812 
 111.3,193.812 112.202,193.812 113.104,193.812 114.006,193.812 114.908,193.812 115.81,193.812 116.711,193.812 117.613,193.812 118.515,193.812 119.417,193.812 
 120.319,193.812 121.221,193.812 122.123,193.812 123.025,193.812 123.927,193.812 124.829,193.812 125.731,193.812 126.633,193.812 127.535,193.812 128.437,193.812 
 129.339,193.812 130.241,193.812 131.143,193.812 132.045,193.812 132.947,193.812 133.849,193.812 134.751,193.812 135.652,193.812 136.554,193.812 137.456,193.812 
 138.358,193.812 139.26,193.812 140.162,193.812 141.064,193.812 141.966,193.812 142.868,193.812 143.77,193.812 144.672,193.812 145.574,193.812 146.476,193.812 
 147.378,193.812 148.28,193.812 149.182,193.812 150.084,193.812 150.986,193.812 151.888,193.812 152.79,193.812 153.692,193.812 154.593,193.812 155.495,193.812 
 156.397,193.812 157.299,193.812 158.201,193.812 159.103,193.812 160.005,193.812 160.907,193.812 161.809,193.812 162.711,193.812 163.613,193.812 164.515,193.812 
 165.417,193.812 166.319,193.812 167.221,193.812 168.123,193.812 169.025,193.812 169.927,193.812 170.829,193.812 171.731,193.812 172.633,193.812 173.534,193.812 
 174.436,193.812 175.338,193.812 176.24,193.812 177.142,193.812 178.044,193.812 178.946,193.812 179.848,193.812 180.75,193.812 181.652,193.812 182.554,193.812 
 183.456,193.812 184.358,193.812 185.26,193.812 186.162,193.812 187.064,193.812 187.966,193.812 188.868,193.812 189.77,193.812 190.672,193.812 191.574,193.812 
 192.475,193.812 193.377,193.812 194.279,193.812 195.181,193.812 196.083,193.812 196.985,193.812 197.887,193.812 198.789,193.812 199.691,193.812 200.593,193.812 
 201.495,193.812 202.397,193.812 203.299,193.812 204.201,193.812 205.103,193.812 206.005,193.812 206.907,193.812 207.809,193.812 208.711,193.812 209.613,193.812 
 210.515,193.812 211.416,193.812 212.318,193.812 213.22,193.812 214.122,193.812 215.024,193.812 215.926,193.812 216.828,193.812 217.73,193.812 218.632,193.812 
 219.534,193.812 220.436,361.442 221.338,336.211 222.24,321.452 223.142,310.98 224.044,302.858 224.946,296.221 225.848,290.61 226.75,285.749 227.652,281.462 
 228.554,277.627 229.456,274.158 230.357,270.99 231.259,268.077 232.161,265.379 233.063,262.868 233.965,260.519 234.867,258.312 235.769,256.231 236.671,254.263 
 237.573,252.396 238.475,250.62 239.377,248.927 240.279,247.309 241.181,245.76 242.083,244.274 242.985,242.846 24

In [ ]:
log(0.1)